In [2]:
import numpy as np
import pandas as pd
import calendar
import copy
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,MinMaxScaler
import calendar
import datetime
from datetime import datetime
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


/usr/local/anaconda3/lib/python3.5/site-packages/sklearn/utils/fixes.py:313: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  _nan_object_mask = _nan_object_array != _nan_object_array


In [5]:
cardf= pd.read_csv('../../Data/cardata.csv')
cardf.head()


,Month,WeekOfMonth,DayOfWeek,Make,AccidentArea,DayOfWeekClaimed,MonthClaimed,WeekOfMonthClaimed,Sex,MaritalStatus,...,AgeOfVehicle,AgeOfPolicyHolder,PoliceReportFiled,WitnessPresent,AgentType,NumberOfSuppliments,AddressChange-Claim,NumberOfCars,Year,BasePolicy
0,Dec,5,Wednesday,Honda,Urban,Tuesday,Jan,1,Female,Single,...,3 years,26 to 30,No,No,External,none,1 year,3 to 4,1994,Liability
1,Jan,3,Wednesday,Honda,Urban,Monday,Jan,4,Male,Single,...,6 years,31 to 35,Yes,No,External,none,no change,1 vehicle,1994,Collision
2,Oct,5,Friday,Honda,Urban,Thursday,Nov,2,Male,Married,...,7 years,41 to 50,No,No,External,none,no change,1 vehicle,1994,Collision
3,Jun,2,Saturday,Toyota,Rural,Friday,Jul,1,Male,Married,...,more than 7,51 to 65,Yes,No,External,more than 5,no change,1 vehicle,1994,Liability
4,Jan,5,Monday,Honda,Urban,Tuesday,Feb,2,Female,Single,...,5 years,31 to 35,No,No,External,none,no change,1 vehicle,1994,Collision


In [6]:
#define the fraud indicators
#########indicator1:
"""
if( Day:Policy-Accident == 1-7 and Day:Policy-Claim== 15-30):
    fraud;
    
if(Age-of-Vehicle == new && Age:=16-17 && Vehicle Price == more than 69000):
    fraud
if(Age-of-Vehicle== 7 and more than 7 and Address-Change== under 6 months):
    fraud
    
"""

#new data frame
columnLabel= list(cardf.columns.values)
carDataIndicatorBased= pd.DataFrame(columns=columnLabel)
carDataAfterPassingIndicator= pd.DataFrame(columns=columnLabel)

carSize= cardf.shape[0]
index1= 0
index2= 0
for i in range(carSize):
    if(cardf['Days:Policy-Accident'][i]== '1 to 7' and cardf['Days:Policy-Claim'][i]=='15 to 30'):
        carDataIndicatorBased.loc[index1]= cardf.loc[i]
        index1= index1 + 1
    elif(cardf['AgeOfVehicle'][i]== 'new' and cardf['AgeOfPolicyHolder'][i]=='new' and cardf['VehiclePrice'][i]== 'more than 69,000'):
        carDataIndicatorBased.loc[index1]== cardf.loc[i]
        index1= index1 + 1
    elif((cardf['AgeOfVehicle'][i]=='7 years' or cardf['AgeOfVehicle'][i]=='more than 7') and cardf['AddressChange-Claim'][i]=='under 6 months'):
        carDataIndicatorBased.loc[index1]= cardf.loc[index1]
        index1 = index1 + 1
    else:
        carDataAfterPassingIndicator.loc[index2]= cardf.loc[i]
        index2 = index2 + 1


In [14]:
carDataAfterPassingIndicator.shape

(15411, 33)

In [7]:
carDataIndicatorBased.head()

,Month,WeekOfMonth,DayOfWeek,Make,AccidentArea,DayOfWeekClaimed,MonthClaimed,WeekOfMonthClaimed,Sex,MaritalStatus,...,AgeOfVehicle,AgeOfPolicyHolder,PoliceReportFiled,WitnessPresent,AgentType,NumberOfSuppliments,AddressChange-Claim,NumberOfCars,Year,BasePolicy
0,Dec,5.0,Wednesday,Honda,Urban,Tuesday,Jan,1.0,Female,Single,...,3 years,26 to 30,No,No,External,none,1 year,3 to 4,1994.0,Liability
1,Jan,3.0,Wednesday,Honda,Urban,Monday,Jan,4.0,Male,Single,...,6 years,31 to 35,Yes,No,External,none,no change,1 vehicle,1994.0,Collision
2,Oct,5.0,Friday,Honda,Urban,Thursday,Nov,2.0,Male,Married,...,7 years,41 to 50,No,No,External,none,no change,1 vehicle,1994.0,Collision
3,Jan,4.0,Monday,Honda,Urban,Thursday,Feb,1.0,Male,Single,...,6 years,26 to 30,No,No,External,none,no change,1 vehicle,1994.0,Collision
4,Apr,4.0,Thursday,Honda,Urban,Thursday,May,1.0,Male,Married,...,7 years,36 to 40,No,No,External,none,4 to 8 years,2 vehicles,1994.0,All Perils
